In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sunpy.time import parse_time
from sklearn import preprocessing

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *

import os
import shutil
import datetime
import sys
from pathlib import Path
from PIL import Image, ImageDraw, ImageEnhance, ImageFont
from matplotlib import cm
import cv2
import imageio

sys.path.append(os.path.join(Path.cwd(), 'utils'))
sys.path.append(os.path.join(Path.cwd(), 'models'))

from utils.region_detector import *
from utils.im_utils import *
from utils.data_augmentation import *
from models.bidirectional_convlstm_model import *

In [2]:
DATA_DIR = '../data'
VIDEOS_DIR = './videos'
FRAMES_DIR = './frames'
FRAMES_MARKED_DIR = './frames/frames_marked'
CUTOUT_FRAMES_MARKED = './frames/cutout_frames_marked/'
LSTM_CHECKPOINTS_DIR = './checkpoints/lstm_checkpoints'
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
def delete_files(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [4]:
# returns a formatted file name of the closest AIA data file to the given datetime

def GetClosestDataFileByDate(dt):
    AIA_data_date = f'{dt.year}{dt.month:02d}{dt.day:02d}'
    tmp_dt = dt
    minute = 0
    minute=GetClosestMultiple(tmp_dt.minute, 6)
    AIA_data_time = f'{tmp_dt.hour:02d}{minute:02d}'
    AIA_data_filename = f'AIA{AIA_data_date}_{AIA_data_time}_0094.npz'
    
    return AIA_data_filename

In [5]:
def GetAIAPathAtTime(dt):
    dt_data_dir = os.path.join(DATA_DIR, f'{dt.year}/{dt.month:02d}/{dt.day:02d}')
    closest_data_file = GetClosestDataFileByDate(dt)
    file_path = os.path.join(dt_data_dir, closest_data_file)
    if not os.path.exists(file_path):
        raise FileNotFoundError
    
    return file_path

In [6]:
def GetFilepathsBetweenDates(start_datetime, end_datetime, cadence=6):
    filepaths = []
    loop_date = start_datetime
    
    while(loop_date < end_datetime):
        closest_filepath = None
        try:
            closest_filepath = GetAIAPathAtTime(loop_date)
        except(FileNotFoundError):
            loop_date = loop_date + datetime.timedelta(minutes=cadence)
            continue
        filepaths.append(closest_filepath)
        loop_date = loop_date + datetime.timedelta(minutes=cadence)
    
    return filepaths

In [7]:
def key_func(x):
    return int(x.split('.')[-2].split('/')[-1])

In [8]:
def WriteImages(filepaths):
    frames = [Image.fromarray(np.uint8(np.load(x)['x'])) for x in filepaths]
    for i, frame in enumerate(frames):
        frame.save(f'{FRAMES_DIR}/{i}.png')

In [9]:
def GetCleanAIAFilename(filename):
    return filename.rsplit('/', 1)[-1].rsplit('.', 1)[0]

In [10]:
def PosToClass(pos):
    if pos == 0:
        return 'N'
    # elif pos == 1:
    #     return 'N'
    elif pos == 1:
        return 'M'
    elif pos == 2:
        return 'X'

In [11]:
def GetImageCutouts(image, cutout_size=64, stride=32):
    width, height = image.shape[0] , image.shape[1]
    h_pass, v_pass = height//cutout_size, width//cutout_size
    cutouts = []
    for i in range(0, height-cutout_size, stride):
        for j in range(0, width-cutout_size, stride):
            cutouts.append(image[i:i+cutout_size, j:j+cutout_size])
    
    return np.array(cutouts)

In [12]:
def WritePredictedCutouts(save_dir, filepaths, model, image_size=64, sequence_length=5):
    delete_files(save_dir)
    images_raw = [np.load(x)['x'] for x in filepaths]
    images_raw_diff = [abs(images_raw[x]-images_raw[x-1]) for x in range(1, len(images_raw))]
    
    for i in range(232, len(images_raw_diff)):
        print('--------------------------------------------------------------------------------------')
        print(f'from: {GetCleanAIAFilename(filepaths[i-sequence_length])} to: {GetCleanAIAFilename(filepaths[i+1])}')
        past_images = images_raw_diff[i-sequence_length:i]
        past_images = np.array(past_images)
        # shape = 5, 64, 64
        past_images_cutouts = np.array([GetImageCutouts(x) for x in past_images])
        # shape = 5, N, 64, 64
        past_images_cutouts = np.array([past_images_cutouts[:, x] for x in range(past_images_cutouts.shape[1])])
        # past_images_cutouts = np.reshape(past_images_cutouts, (past_images_cutouts.shape[1], past_images_cutouts.shape[0], past_images_cutouts.shape[2], past_images_cutouts.shape[3]))
        print(past_images_cutouts.shape)
        # shape = N, 5, 64, 64
        predictions = []
        for idx, c in enumerate(past_images_cutouts):
            # print(f'predicting cutout {i}')
            cutouts = np.expand_dims(c, 0)
            cutouts = np.expand_dims(cutouts, 4)
            # shape = 1, 5, 64, 64, 1
            predictions.append(model.predict(cutouts, verbose=0))
        predictions = np.array(predictions)
        # print(predictions)
        predicted_classes = np.array([x.argmax() for x in predictions])
        flare_cutout_indeces = np.where(predicted_classes > 0)[0]
        print(flare_cutout_indeces)
        flare_center_coords = [(int(x/14)*32, int(x%14)*32) for x in flare_cutout_indeces]
        
        past_images_pil = [Image.fromarray(np.uint8(images_raw[i-x])).convert("RGBA") for x in range(sequence_length)]
        past_images_raw = [images_raw[i-x] for x in range(sequence_length)]
        last_image_pil = past_images_pil[0]
        print(flare_center_coords)
        
        for center_coord in flare_center_coords:
            tl, br = (center_coord[1], center_coord[0]), (center_coord[1]+64, center_coord[0]+64)
            print(f'{tl} {br}')
            draw = ImageDraw.Draw(last_image_pil)
            draw.rectangle((tl, br), outline="red")
            # draw.rectangle(((100, 100), (200, 200)), outline="red")
        # draw = ImageDraw.Draw(last_image_pil)
        filename = filepaths[i].rsplit('/')[-1].rsplit('.', 1)[0]
        # draw.text((12, 490),filename,(255,255,255))
        # draw.text((400, 490),PosToClass(pos),(255,255,255))
        last_image_pil.save(f'{save_dir}/{i}.png', "PNG")
        
#         for pc in predicted_classes:
            

In [13]:
def WritePredictedImages(save_dir, filepaths, model, image_size=64, sequence_length=5, threshold=0.9, rect_size=64):
    delete_files(save_dir)
    images_raw = [np.load(x)['x'] for x in filepaths]
    images_raw_diff = [abs(images_raw[x]-images_raw[x-1]) for x in range(1, len(images_raw))]
    images = [cv2.resize(x, (image_size, image_size), interpolation = cv2.INTER_AREA) for x in images_raw]
    images_diff = [cv2.resize(x, (image_size, image_size), interpolation = cv2.INTER_AREA) for x in images_raw_diff]
    for i in range(sequence_length, len(images_diff)):
        print('--------------------------------------------------------------------------------------')
        print(f'from: {GetCleanAIAFilename(filepaths[i-sequence_length])} to: {GetCleanAIAFilename(filepaths[i+1])}')
        past_images = images_diff[i-sequence_length:i]
        past_images = np.array(past_images)
        past_images = np.expand_dims(past_images, 0)
        past_images = np.expand_dims(past_images, 4)
        # now the shape is 1, 12, 64, 64, 1 of full sun images
        prediction = model.predict(past_images, verbose=0)
        pos = prediction[0].argmax()
        # print(f'{prediction} CLASS: {PosToClass(pos)}')
        # print('--------------------------------------------------------------------------------------')
        past_images_pil = [Image.fromarray(np.uint8(images_raw[i-x])).convert("RGBA") for x in range(sequence_length)]
        past_images_raw = [images_raw[i-x] for x in range(sequence_length)]
        last_image_pil = past_images_pil[0]
        print(prediction)
        print(pos)
        if pos > 0:
        # if prediction > threshold:
            last_image_raw = past_images_raw[0]
            center_coord = GetImageTopNRegionsCoords(last_image_raw, 1)[0]
            tl, br = (center_coord[1]-rect_size//2, center_coord[0]-rect_size//2), (center_coord[1]+rect_size//2, center_coord[0]+rect_size//2)
            draw = ImageDraw.Draw(last_image_pil)
            draw.rectangle((tl, br), outline="red")
        draw = ImageDraw.Draw(last_image_pil)
        filename = filepaths[i].rsplit('/')[-1].rsplit('.', 1)[0]
        draw.text((12, 490),filename,(255,255,255))
        draw.text((400, 490),PosToClass(pos),(255,255,255))
        last_image_pil.save(f'{save_dir}/{i}.png', "PNG")

In [14]:
def WriteVideo(frames_dir, video_name):
    paths = []
    for subdir, dirs, files in os.walk(frames_dir):
        for f in files:
            if f.rsplit('.', 1)[-1] == 'png':
                paths.append(os.path.join(subdir, f))
        break
    paths = sorted(paths, key=key_func)
    writer = imageio.get_writer(f'{VIDEOS_DIR}/{video_name}.mp4', fps=8)
    for file in paths:
        im = imageio.imread(file)
        if im.shape != (512, 512, 4):
            print(im.shape)
            continue
        writer.append_data(im)
    writer.close()

In [15]:
batch_size, sequence_length, image_size, num_classes = 64, 6, 64, 3
start_datetime, end_datetime = datetime.datetime(2016, 1, 1), datetime.datetime(2016, 1, 3)
filepaths = GetFilepathsBetweenDates(start_datetime, end_datetime)
# model = tf.keras.models.load_model('./best_model.h5')
model = ConvLSTMModelAllClass(batch_size, image_size, sequence_length-1, num_classes)
model.load_weights(f'{LSTM_CHECKPOINTS_DIR}/full_image_duringflare_nmx_leftout2013_bidirectional_convlstm')
WritePredictedImages(FRAMES_MARKED_DIR, filepaths, model, image_size=64, sequence_length=sequence_length-1, threshold=0.9, rect_size=64)
# WritePredictedCutouts(CUTOUT_FRAMES_MARKED, filepaths, model, image_size=64, sequence_length=sequence_length-1)
# video_title = f'full_image_{parse_time(start_datetime, precision=0).fits}~{parse_time(end_datetime, precision=0).fits}'
video_title='full_image_2013_preflare_cadence12'
WriteVideo(FRAMES_MARKED_DIR, video_title)

(None, 5, 64, 64, 1)


2022-09-08 12:54:34.919916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 12:54:35.477008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 12:54:35.478064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 12:54:35.500548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

--------------------------------------------------------------------------------------
from: AIA20160101_0000_0094 to: AIA20160101_0036_0094


2022-09-08 12:55:30.816363: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-09-08 12:55:35.905492: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-08 12:55:35.918826: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-08 12:55:35.918845: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-09-08 12:55:35.919549: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-08 12:55:35.919706: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-09-08 12:55:39.748801: I tensorflow/stream_executor/cuda/c

[[9.8849469e-01 1.1504978e-02 3.4737067e-07]]
0
--------------------------------------------------------------------------------------
from: AIA20160101_0006_0094 to: AIA20160101_0042_0094
[[9.8803627e-01 1.1963342e-02 3.7694346e-07]]
0
--------------------------------------------------------------------------------------
from: AIA20160101_0012_0094 to: AIA20160101_0048_0094
[[9.8696101e-01 1.3038575e-02 4.5133427e-07]]
0
--------------------------------------------------------------------------------------
from: AIA20160101_0018_0094 to: AIA20160101_0054_0094
[[9.8698246e-01 1.3017053e-02 4.5034230e-07]]
0
--------------------------------------------------------------------------------------
from: AIA20160101_0024_0094 to: AIA20160101_0100_0094
[[9.8719436e-01 1.2805141e-02 4.3596879e-07]]
0
--------------------------------------------------------------------------------------
from: AIA20160101_0030_0094 to: AIA20160101_0106_0094
[[9.9896443e-01 1.0355668e-03 2.0439390e-09]]
0
-------

/tmp/ipykernel_53859/1067415504.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imageio.imread(file)


In [307]:
model = ConvLSTMModelAllClass(batch_size, image_size, sequence_length)
model.load_weights(f'{LSTM_CHECKPOINTS_DIR}/cutout_image_duringflare_bidirectional_convlstm')
# c = WritePredictedCutouts(CUTOUT_FRAMES_MARKED, filepaths, model, image_size=64, sequence_length=5)
video_title = f'full_image_{parse_time(start_datetime, precision=0).fits}~{parse_time(end_datetime, precision=0).fits}'
WriteVideo(CUTOUT_FRAMES_MARKED, video_title)

(None, 5, 64, 64, 1)


/tmp/ipykernel_19415/2494019916.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imageio.imread(file)


In [22]:
def CreateAIAVideo(start_datetime, end_datetime):
    # filepaths = GetFilepathsBetweenDates(start_datetime, end_datetime)
    # WriteImages(filepaths)
    WriteVideo(FRAMES_DIR)

In [ ]:
def DrawRectangle(img, top_left, bottom_right, save_dir):
    result = img.copy()
    result = cv2.rectangle(result, coord1, coord2, color=(0, 0, 255), thickness=3)
    cv2.imwrite(save_dir, result)

In [ ]:
def CreateAIAVideoPrediction(start_datetime, end_datetime, model):
    filepaths = GetFilepathsBetweenDates(start_datetime, end_datetime)
    return WritePredictedImages(filepaths, model)

In [ ]:
start_datetime, end_datetime = datetime.datetime(2016, 1, 1), datetime.datetime(2016, 1, 3)
model = ConvLSTMModel(64)
model.load_weights(f'{LSTM_CHECKPOINTS_DIR}/conv_lstm_trial_5')
# end_model = ConvLSTMModel(64)
# end_model.load_weights(f'{LSTM_CHECKPOINTS_DIR}/conv_lstm_trial_end_full')
delete_files(FRAMES_MARKED_DIR)
CreateAIAVideoPrediction(start_datetime, end_datetime, model)
WriteVideo(FRAMES_MARKED_DIR)